# Hyperparameter tuning with XGBoost, Ray Tune, Hyperopt and Optuna

## Introduction


In this post we are going to demonstrate how we can speed up hyperparameter tuning with:

1) Bayesian optimization tuning algos like HyperOpt and Optuna, running on…

2) the [Ray](https://ray.io/) distributed ML framework, with a [unified API to many hyperparameter search algos](https://medium.com/riselab/cutting-edge-hyperparameter-tuning-with-ray-tune-be6c0447afdf) with early stopping and…

3) a distributed cluster of cloud instances for even more speedup.

### Outline:
- Overview of hyperparameter tuning
- Baseline linear regression with no hyperparameters
- ElasticNet with L1 and L2 regularization using ElasticNetCV hyperparameter optimization
- ElasticNet with GridSearchCV hyperparameter optimization
- XGBoost: sequential grid search over hyperparameter subsets with early stopping 
- XGBoost: with HyperOpt and Optuna search algorithms
- LightGBM: with HyperOpt and Optuna search algorithms
- XGBoost: HyperOpt on a Ray cluster
- LightGBM: HyperOpt on a Ray cluster
- Concluding remarks
18735

But first, here are results on the Ames housing data set, predicting Iowa home prices:

| ML Algo           | Hyperparameter search algo   | CV Error (RMSE in $)  | Time     |
|-------------------|------------------------------|-----------------------|----------|
| XGB               | Sequential Grid Search       | 18783                |   36:09  |
| XGB               | HyperOpt (128 samples)       | 18770                | 53:27  |
| LightGBM          | HyperOpt (256 samples)       |                 |    |
| XGB               | Optuna (256 samples)         | 18735                | 2:46:44 |
| LightGBM          | Optuna                       | 18612                | 0:47:31 |
| XGB               | Optuna - 16-instance cluster | 18770                |   14:23  |
| LightGBM          | Optuna - 16-instance cluster | 18612                |    4:22  |
| Baseline:         |                              |
| Linear Regression | --                           | 18192                |   0:01s  |
| ElasticNet        | ElasticNetCV (Grid Search)   | 18122                |   0:02s  |          
| ElasticNet        | GridSearchCV                 | 18061                |   0:05s  |          

We see both speedup and RMSE improvement when using HyperOpt and Optuna, and the cluster. But our feature engineering was quite good and our simple linear model still outperforms boosting. (Not shown, SVR and KR are high-performing and an ensemble improves over all individual algos)


## Hyperparameter Tuning Overview

Here are [the principal approaches to hyperparameter tuning](https://en.wikipedia.org/wiki/Hyperparameter_optimization)

- Grid search: given a finite set of discrete values for each hyperparameter, exhaustively cross-validate all combinations

- Random search: given a discrete or continuous distribution for each hyperparameter, randomly sample from the joint distribution. Generally [more efficient than exhaustive grid search.](https://dl.acm.org/doi/10.5555/2188385.2188395 ) 

- Bayesian optimization: update the search space as you go based on outcomes of prior searches.

- Gradient-based optimization: attempt to estimate the gradient of the CV metric with respect to the hyperparameter and ascend/descend the gradient.

- Evolutionary optimization: sample the search space, discard combinations with poor metrics, and genetically evolve new combinations to try based on the successful combinations.

- Population-based: A method of performing hyperparameter optimization at the same time as training.

In this post we focus on Bayesian optimization with HyperOpt and Optuna. What is Bayesian optimization? When we perform a grid search, the search space can be considered a prior belief that the best hyperparameter vector is in the search space, and the combinations have equal probability of being the best combination. So we try them all and pick the best one.

Perhaps we might do two passes of grid search. After an initial search on a broad, coarsely spaced grid, we might do a deeper dive in a smaller area around the best metric from the first pass, with a more finely-spaced grid. In Bayesian terminology we updated our prior belief.

Bayesian optimization first samples randomly, e.g. 30 combinations, and computes the cross-validation metric for each combination. Then the algorithm updates the distribution it samples from, so it is more likely to sample combinations near the good metrics, and less likely to sample combinations near the poor metrics. As it continues to sample, it continues to update the search distribution based on the metrics it finds.

Early stopping may also highly beneficial: often we can discard a combination without fully training it. In this post we use [ASHA](https://arxiv.org/abs/1810.05934). 

We use 4 regression algorithms:
- LinearRegression: baseline with no hyperparameters
- ElasticNet: Linear regression with L1 and L2 regularization (2 hyperparameters).
- XGBoost
- LightGBM

We use 5 approaches :
- *Native CV*: In sklearn if an algo has hyperparameters it will often have an xxxCV version which performs automated hyperparameter tuning over a search space with specified kfolds.
- *GridSearchCV*: Abstracts CV for any sklearn algo, running multithreaded trials over specified folds. 
- *Manual sequential grid search*: What we typically do with XGBoost, which doesn't play well with GridSearchCV and has too many hyperparameters to tune in one pass.
- *Ray on local machine*: HyperOpt and Optuna with early stopping.
- *Ray on cluster*: Additionally scale out to run a single hyperparameter optimization task over many instances.

We use data from the Ames Housing Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques . The original data has 79 raw features. The data we will use has 100 features with a fair amount of feature engineering from [my own attempt at modeling](https://github.com/druce/iowa), which was in the top 5% or so when I submitted it to Kaggle.

### Further reading: 
 - [Hyper-Parameter Optimization: A Review of Algorithms and Applications](https://arxiv.org/abs/2003.05689) Tong Yu, Hong Zhu (2020)
 - [Hyperparameter Search in Machine Learning](https://arxiv.org/abs/1502.02127v2), Marc Claesen, Bart De Moor (2015)
 - [Hyperparameter Optimization](https://link.springer.com/chapter/10.1007/978-3-030-05318-5_1), Matthias Feurer, Frank Hutter (2019) 

In [1]:
from itertools import product
from datetime import datetime, timedelta
import os
import random
import string

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline

#!conda install -y -c conda-forge  xgboost 
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

import lightgbm
from lightgbm import LGBMRegressor

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger

# pip install hyperopt
# pip install optuna

import wandb
os.environ['WANDB_NOTEBOOK_NAME']='hyperparameter_optimization.ipynb'

print(datetime.now())

print ("%-20s %s"% ("numpy", np.__version__))
print ("%-20s %s"% ("pandas", pd.__version__))
print ("%-20s %s"% ("sklearn", sklearn.__version__))
print ("%-20s %s"% ("xgboost", xgboost.__version__))
print ("%-20s %s"% ("lightgbm", lightgbm.__version__))
print ("%-20s %s"% ("ray", ray.__version__))


2020-10-14 16:30:05.784018
numpy                1.19.1
pandas               1.1.3
sklearn              0.23.2
xgboost              1.2.0
lightgbm             2.3.0
ray                  1.0.0


In [2]:
# set seed for reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)


In [3]:
def get_random_tag(length):
    """random tag for experiments"""
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    return result_str.upper()

get_random_tag(8)

'CVJOZKZO'

In [4]:
# import train data
df = pd.read_pickle('df_train.pickle')

response = 'SalePrice'
predictors = ['YearBuilt',
              'BsmtFullBath',
              'FullBath',
              'KitchenAbvGr',
              'GarageYrBlt',
              'LotFrontage',
              'MasVnrArea',
              '1stFlrSF',
              'GrLivArea',
              'GarageArea',
              'WoodDeckSF',
              'PorchSF',
              'AvgBltRemod',
              'FireBathRatio',
              'TotalSF x OverallQual x OverallCond',
              'AvgBltRemod x Functional x TotalFinSF',
              'Functional x OverallQual',
              'KitchenAbvGr x KitchenQual',
              'GarageCars x GarageYrBlt',
              'GarageQual x GarageCond x GarageCars',
              'HeatingQC x Heating',
              'monthnum',
              'log_YearBuilt',
              'log_LotArea',
              'log_TotalFinSF',
              'log_GarageRatio',
              'log_TotalSF x OverallQual x OverallCond',
              'log_TotalSF x OverallCond',
              'log_AvgBltRemod x TotalFinSF',
              'sq_2ndFlrSF',
              'sq_BsmtFinSF',
              'sq_BsmtFinSF x BsmtQual',
              'sq_BsmtFinSF x BsmtBath',
              'BldgType_4',
              'BsmtExposure_1',
              'BsmtExposure_4',
              'BsmtFinType1_1',
              'BsmtFinType1_2',
              'BsmtFinType1_4',
              'BsmtFinType1_5',
              'BsmtFinType1_6',
              'CentralAir_0',
              'CentralAir_1',
              'Condition1_1',
              'Condition1_3',
              'ExterCond_2',
              'ExterQual_2',
              'Exterior1st_4',
              'Exterior1st_5',
              'Exterior1st_10',
              'Fence_0',
              'Fence_2',
              'Foundation_1',
              'Foundation_5',
              'GarageCars_1',
              'GarageFinish_2',
              'GarageFinish_3',
              'GarageType_2',
              'HouseStyle_2',
              'KitchenQual_4',
              'LotConfig_0',
              'LotConfig_4',
              'MSSubClass_30',
              'MSSubClass_70',
              'MSZoning_0',
              'MSZoning_1',
              'MSZoning_4',
              'MasVnrType_2',
              'MasVnrType_3',
              'MoSold_1',
              'MoSold_5',
              'MoSold_6',
              'MoSold_11',
              'Neighborhood_3',
              'Neighborhood_4',
              'Neighborhood_5',
              'Neighborhood_10',
              'Neighborhood_11',
              'Neighborhood_16',
              'Neighborhood_17',
              'Neighborhood_19',
              'Neighborhood_22',
              'Neighborhood_24',
              'OverallCond_7',
              'OverallQual_5',
              'OverallQual_6',
              'OverallQual_7',
              'OverallQual_9',
              'PavedDrive_0',
              'PavedDrive_2',
              'SaleCondition_1',
              'SaleCondition_2',
              'SaleCondition_5',
              'SaleType_4',
              'BedroomAbvGr_1',
              'BedroomAbvGr_4',
              'BedroomAbvGr_5',
              'HalfBath_1',
              'TotalBath_1.0',
              'TotalBath_2.5']

display(df[predictors].head())
display(df[[response]].head())


,YearBuilt,BsmtFullBath,FullBath,KitchenAbvGr,GarageYrBlt,LotFrontage,MasVnrArea,1stFlrSF,GrLivArea,GarageArea,...,SaleCondition_1,SaleCondition_2,SaleCondition_5,SaleType_4,BedroomAbvGr_1,BedroomAbvGr_4,BedroomAbvGr_5,HalfBath_1,TotalBath_1.0,TotalBath_2.5
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1,2,1,7,65.0,196.0,856,1710,548.0,...,0,0,0,1,0,0,0,1,0,0
2,34,0,2,1,34,80.0,0.0,1262,1262,460.0,...,0,0,0,1,0,0,0,0,0,1
3,9,1,2,1,9,68.0,162.0,920,1786,608.0,...,0,0,0,1,0,0,0,1,0,0
4,95,1,1,1,12,60.0,0.0,961,1717,642.0,...,1,0,0,1,0,0,0,0,0,0
5,10,1,2,1,10,84.0,350.0,1145,2198,836.0,...,0,0,0,1,0,1,0,1,0,0


,SalePrice
Id,
1,12.247699
2,12.109016
3,12.317171
4,11.849405
5,12.429220


In [5]:
# we are training on a response which is the log of 1 + the sale price
# transform prediction back to original basis with expm1 and evaluate vs. original

def evaluate(y_train, y_pred_train, y_test, y_pred_test):
    """evaluate in train_test split"""
    print('Train RMSE', np.sqrt(mean_squared_error(np.expm1(y_train), np.expm1(y_pred_train))))
    print('Train R-squared', r2_score(np.expm1(y_train), np.expm1(y_pred_train)))
    print('Train MAE', mean_absolute_error(np.expm1(y_train), np.expm1(y_pred_train)))
    print()
    print('Test RMSE', np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred_test))))
    print('Test R-squared', r2_score(np.expm1(y_test), np.expm1(y_pred_test)))
    print('Test MAE', mean_absolute_error(np.expm1(y_test), np.expm1(y_pred_test)))

MEAN_RESPONSE=df[response].mean()
def cv_to_raw(cv_val, mean_response=MEAN_RESPONSE):
    """convert log1p rmse to underlying SalePrice error"""
    # MEAN_RESPONSE assumes folds have same mean response, which is true in expectation but not in each fold
    # we can also pass the actual response for each fold
    # but we're usually looking to consistently convert the log value to a more meaningful unit
    return np.expm1(mean_response+cv_val) - np.expm1(mean_response)

In [6]:
# always use same k-folds for reproducibility
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOMSTATE)


## Baseline linear regression
- Raw CV RMSE 18191.9791

In [7]:
%%time
# Tune lr search space for alphas and l1_ratio
print("LinearRegression")

print(len(predictors), "predictors")

lr = LinearRegression()

# evaluate using kfolds
scores = -cross_val_score(lr, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


LinearRegression
100 predictors

Log1p CV RMSE 0.1037 (STD 0.0099)
Raw CV RMSE 18192 (STD 1839)
CPU times: user 64.1 ms, sys: 73.2 ms, total: 137 ms
Wall time: 1.08 s


## Native Sklearn xxxCV
- LogisticRegressionCV, LassoCV, RidgeCV, ElasticNetCV, etc.
- Test many hyperparameters in parallel with multithreading
- Note improvement vs. LinearRegression due to controlling overfitting
- RMSE $18103
- Time 5s


In [8]:
%%time
# Tune elasticnet search space for alphas and L1_ratio
# predictor selection used to create the training set used lasso
# so l1 parameter is close to 0
# could use ridge (eg elasticnet with 0 L1 regularization)
# but then only 1 param, more general and useful to do this with elasticnet
print("ElasticnetCV")

# make pipeline
# with regularization must scale predictors
elasticnetcv = make_pipeline(RobustScaler(),
                             ElasticNetCV(max_iter=100000, 
                                          l1_ratio=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                          alphas=np.logspace(-4, -2, 9),
                                          cv=kfolds,
                                          n_jobs=-1,
                                          verbose=1,
                                         ))

#train and get hyperparams
elasticnetcv.fit(df[predictors], df[response])
l1_ratio = elasticnetcv._final_estimator.l1_ratio_
alpha = elasticnetcv._final_estimator.alpha_
print('l1_ratio', l1_ratio)
print('alpha', alpha)

# evaluate using kfolds on full dataset
# I don't see API to get CV error from elasticnetcv, so we use cross_val_score
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=10000)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


ElasticnetCV


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
...........................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.5s
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

l1_ratio 0.01
alpha 0.0031622776601683794

Log1p CV RMSE 0.1030 (STD 0.0109)
Raw CV RMSE 18061 (STD 2008)
CPU times: user 7.31 s, sys: 5.64 s, total: 13 s
Wall time: 2.06 s


## GridSearchCV
- Useful for algos with no native multithreaded xxxCV
- Test many hyperparameter combinations in parallel with multithreading
- Similar result vs ElasticNetCV, not exact, need more research as to why


In [9]:
%%time
gs = make_pipeline(RobustScaler(),
                   GridSearchCV(ElasticNet(max_iter=100000),
                                param_grid={'l1_ratio': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                            'alpha': np.logspace(-4, -2, 9),
                                           },
                                scoring='neg_mean_squared_error',
                                refit=True,
                                cv=kfolds,
                                n_jobs=-1,
                                verbose=1
                               ))

# do cv using kfolds on full dataset
print("\nCV on full dataset")
gs.fit(df[predictors], df[response])
print('best params', gs._final_estimator.best_params_)
print('best score', -gs._final_estimator.best_score_)
l1_ratio = gs._final_estimator.best_params_['l1_ratio']
alpha = gs._final_estimator.best_params_['alpha']
print("Log1p CV RMSE %.06f" % (np.sqrt(-gs._final_estimator.best_score_)))
print("Raw CV RMSE %.0f" % (cv_to_raw(np.sqrt(-gs._final_estimator.best_score_))))

# we shouldn't need to do this
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=100000)
print(elasticnet)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))

# difference in average CV scores reported by GridSearchCV and cross_val_score
# with same alpha, l1_ratio, kfolds
# one reason could be that we used simple average, GridSearchCV is weighted by # of samples per fold?
nsamples = [len(z[1]) for z in kfolds.split(df)]
print("weighted average %.06f" % np.average(scores, weights=nsamples))
# still tiny difference, not sure why, also ElasticSearchCV shows fewer fits, takes less time



CV on full dataset
Fitting 10 folds for each of 117 candidates, totalling 1170 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 656 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 1155 out of 1170 | elapsed:    4.3s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1170 out of 1170 | elapsed:    4.3s finished


best params {'alpha': 0.0031622776601683794, 'l1_ratio': 0.01}
best score 0.010637685614240404
Log1p CV RMSE 0.103139
Raw CV RMSE 18075
ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=100000)

Log1p CV RMSE 0.103003 (STD 0.0109)
Raw CV RMSE 18061 (STD 2008)
weighted average 0.103023
CPU times: user 1.57 s, sys: 378 ms, total: 1.95 s
Wall time: 4.76 s


In [10]:
# roll-our-own CV 
# matches cross_val_score
alpha = 0.0031622776601683794
l1_ratio = 0.01
regressor = ElasticNet(alpha=alpha,
                       l1_ratio=l1_ratio,
                       max_iter=10000)
print(regressor)
cverrors = []
for train_fold, cv_fold in kfolds.split(df): 
    fold_X_train=df[predictors].values[train_fold]
    fold_y_train=df[response].values[train_fold]
    fold_X_test=df[predictors].values[cv_fold]
    fold_y_test=df[response].values[cv_fold]
    regressor.fit(fold_X_train, fold_y_train)
    y_pred_test=regressor.predict(fold_X_test)
    cverrors.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
    
print("%.06f" % np.average(cverrors))
    

ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=10000)
0.103003


## XGBoost Sequential Grid Search 
- XGBoost has native multithreading, CV
- XGBoost has many tuning parameters so a complete grid search has an unreasonable number of combinations
- We tune reduced sets sequentially and use early stopping. 

### Tuning methodology
- Set an initial set of starting parameters
- Do 10-fold CV
- Use early stopping to halt training in each fold if no improvement after eg 100 rounds, pick hyperparameters to minimize average error over kfolds
- Tune sequentially on groups of hyperparameters that don't interact too much between groups to reduce combinations
- Tune max_depth and min_child_weight 
- Tune subsample and colsample_bytree
- Tune alpha, lambda and gamma (regularization)
- Tune learning rate: lower learning rate will need more rounds/n_estimators
- Retrain and evaluate on full dataset with best learning rate and best n_estimators (average stopping point over kfolds)

### Notes
- It doesn't seem possible to get XGBoost early stopping and also use GridSearchCV. GridSearchCV doesn't pass the kfolds in a way that XGboost understands for early stopping
- 2 alternative approaches 
    - use native xgboost .cv which understands early stopping but doesn't use sklearn API (uses DMatrix, not np array or dataframe)
    - use sklearn API and roll our own grid search instead of GridSearchCV (used below)
- XGboost terminology differs from sklearn
    - boost_rounds = n_estimators
    - eta = learning_rate
- parameter reference: https://xgboost.readthedocs.io/en/latest/parameter.html
- training reference: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
- times are wall times on an amazon t2.2xlarge instance  
- to set up environment:
    - `conda create --name hyperparam python=3.8`
    - `conda activate hyperparam`
    - `conda install jupyter`
    - `pip install -r requirements.txt`
- round 1 Wall time: 6min 23s
- round 2 Wall time: 19min 22s
- round 3 Wall time: 5min 30s
- round 4 Wall time: 4min 54s
- total time 36:09
- RMSE 18783.117031

In [11]:
#%%time
# this cell runs a single round
# the full process is 
# set initial XGboost parameters
# remove overrides (search for TODO: in this cell)
# run round 1 and override initial max_depth, min_child_weight based on best values (search for TODO:)
# run round 2 and override subsample and colsample_bytree based on best values
# run round 3 and override reg_alpha, reg_lambda, reg_gamma based on best values
# run round 4 and obtain learning_rate and best n_iterations
# this is not an exhaustive list but a representative list of most important parameters to tune
# see https://xgboost.readthedocs.io/en/latest/parameter.html for all parameters

# in XGBoost > 1.0.2 this seems to give a warning
# Parameters: { early_stopping_rounds } might not be used.
# but early stopping seems to be used correctly

# TODO: refactor as a function that takes a config with search spaces
# returns a fully specified config with the best values
# run 4 times for 4 passes to get a fully cross-validated parameter set.

# initial hyperparams
max_depth = 5
min_child_weight=5
colsample_bytree = 0.5
subsample = 0.5
reg_alpha = 1e-05
reg_lambda = 1
reg_gamma = 0
learning_rate = 0.01

BOOST_ROUNDS=50000   # we use early stopping so make this arbitrarily high
EARLY_STOPPING_ROUNDS=100 # stop if no improvement after 100 rounds

# round 1: tune depth and min_child_weight
max_depths = list(range(1,5))
min_child_weights = list(range(1,5))
gridsearch_params_1 = product(max_depths, min_child_weights)

# round 2: tune subsample and colsample_bytree
subsamples = np.linspace(0.1, 1.0, 10)
colsample_bytrees = np.linspace(0.1, 1.0, 10)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 2 (refined): tune subsample and colsample_bytree
subsamples = np.linspace(0.4, 0.8, 9)
colsample_bytrees = np.linspace(0.05, 0.25, 5)
gridsearch_params_2 = product(subsamples, colsample_bytrees)

# round 3: tune alpha, lambda, gamma
reg_alphas = np.logspace(-3, -2, 3)
reg_lambdas = np.logspace(-2, 1, 4)
reg_gammas = [0]
#reg_gammas = np.linspace(0, 5, 6)
gridsearch_params_3 = product(reg_alphas, reg_lambdas, reg_gammas)

# round 4: learning rate
learning_rates = reversed(np.logspace(-3, -1, 5).tolist())
gridsearch_params_4 = learning_rates

# TODO: remove these overrides to reset the search
# override initial parameters after search
# round 1:
max_depth=2
min_child_weight=2
# # round 2:
subsample=0.60
colsample_bytree=0.05
# # round 3:  
reg_alpha = 0.003162
reg_lambda = 0.1
reg_gamma = 0

def my_cv(df, predictors, response, kfolds, regressor, verbose=False):
    """Roll our own CV over kfolds with early stopping"""
    metrics = []
    best_iterations = []

    for train_fold, cv_fold in kfolds.split(df): 
        fold_X_train=df[predictors].values[train_fold]
        fold_y_train=df[response].values[train_fold]
        fold_X_test=df[predictors].values[cv_fold]
        fold_y_test=df[response].values[cv_fold]
        regressor.fit(fold_X_train, fold_y_train,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      eval_set=[(fold_X_test, fold_y_test)],
                      eval_metric='rmse',
                      verbose=verbose
                     )
        y_pred_test=regressor.predict(fold_X_test)
        metrics.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
        best_iterations.append(xgb.best_iteration)
    return np.average(metrics), np.std(metrics), np.average(best_iterations)

results = []
best_iterations = []

# TODO: iteratively uncomment 1 of the following 4 lines
# for i, (max_depth, min_child_weight) in enumerate(gridsearch_params_1): # round 1
# for i, (subsample, colsample_bytree) in enumerate(gridsearch_params_2): # round 2
# for i, (reg_alpha, reg_lambda, reg_gamma) in enumerate(gridsearch_params_3): # round 3
for i, learning_rate in enumerate(gridsearch_params_4): # round 4

    params = {
        'max_depth': max_depth,
        'min_child_weight': min_child_weight,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'gamma': reg_gamma,
        'learning_rate': learning_rate,
    }
    print("%s params  %3d: %s" % (datetime.strftime(datetime.now(), "%T"), i, params))
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=BOOST_ROUNDS,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        random_state=RANDOMSTATE,    
        verbosity=1,
        n_jobs=-1,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1,      
        **params
    )
    
    metric_rmse, metric_std, best_iteration = my_cv(df, predictors, response, kfolds, xgb, verbose=False)    
    results.append([max_depth, min_child_weight, subsample, colsample_bytree, reg_alpha, reg_lambda, reg_gamma, 
                   learning_rate, metric_rmse, metric_std, best_iteration])
    
    print("%s %3d result mean: %.6f std: %.6f, iter: %.2f" % (datetime.strftime(datetime.now(), "%T"), i, metric_rmse, metric_std, best_iteration))


results_df = pd.DataFrame(results, columns=['max_depth', 'min_child_weight', 'subsample', 'colsample_bytree', 
                               'reg_alpha', 'reg_lambda', 'reg_gamma', 'learning_rate', 'rmse', 'std', 'best_iter']).sort_values('rmse')
results_df


04:45:05 params    0: {'max_depth': 2, 'min_child_weight': 2, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0, 'learning_rate': 0.1}
[04:45:05] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:45:06] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:45:06] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to 

[04:45:28] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:45:29] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:45:32] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

[04:48:41] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:49:04] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:49:27] WARNING: ../src/learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if y

,max_depth,min_child_weight,subsample,colsample_bytree,reg_alpha,reg_lambda,reg_gamma,learning_rate,rmse,std,best_iter
1,2,2,0.6,0.05,0.003162,0.1,0,0.031623,0.105996,0.012395,1134.8
2,2,2,0.6,0.05,0.003162,0.1,0,0.010000,0.106046,0.013608,2888.1
3,2,2,0.6,0.05,0.003162,0.1,0,0.003162,0.106838,0.013188,7783.7
4,2,2,0.6,0.05,0.003162,0.1,0,0.001000,0.107344,0.013132,22413.3
0,2,2,0.6,0.05,0.003162,0.1,0,0.100000,0.109786,0.010449,476.0


In [12]:
# get best hyperparameters from results dataframe
max_depth = int(results_df.iloc[0]['max_depth'])
min_child_weight = results_df.iloc[0]['min_child_weight']
subsample = results_df.iloc[0]['subsample']
colsample_bytree = results_df.iloc[0]['colsample_bytree']
reg_alpha = results_df.iloc[0]['reg_alpha']
reg_lambda = results_df.iloc[0]['reg_lambda']
reg_gamma = results_df.iloc[0]['reg_gamma']
learning_rate = results_df.iloc[0]['learning_rate']
N_ESTIMATORS = int(results_df.iloc[0]['best_iter'])

params = {
    'max_depth': int(max_depth),
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,    
}

print(params)

{'max_depth': 2, 'min_child_weight': 2.0, 'subsample': 0.6, 'colsample_bytree': 0.05, 'reg_alpha': 0.003162, 'reg_lambda': 0.1, 'gamma': 0.0, 'learning_rate': 0.03162277660168379, 'n_estimators': 1134}


In [13]:
%%time
# CV and evaluate with best params and without early stopping

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    booster='gbtree',   
    base_score=0.5, 
    scale_pos_weight=1,        
    **params
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.05, gamma=0.0,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.03162277660168379, max_delta_step=None,
             max_depth=2, min_child_weight=2.0, missing=nan,
             monotone_constraints=None, n_estimators=1134, n_jobs=-1,
             num_parallel_tree=None, random_state=42, reg_alpha=0.003162,
             reg_lambda=0.1, scale_pos_weight=1, subsample=0.6,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106893 (STD 0.0125)
Raw CV RMSE 18783.117031 (STD 2307.2858)
CPU times: user 507 ms, sys: 0 ns, total: 507 ms
Wall time: 5.39 s


In [14]:
# refactor to give ray.tune a single function of hyperparameters to optimize
def my_xgb(config):
    
    # fix these configs to match calling convention
    config['max_depth'] += 2   # hyperopt needs left to start at 0 but we want to start at 2
    config['max_depth'] = int(config['max_depth'])
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=1,
        random_state=RANDOMSTATE,
        booster='gbtree',   
        base_score=0.5, 
        scale_pos_weight=1, 
        **config,
    )
    scores = np.sqrt(-cross_val_score(xgb, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}


In [15]:
config = {
    'max_depth': max_depth-2,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators': N_ESTIMATORS,
}

xgb = my_xgb(config)

print(xgb)
# same mse

Session not detected. You should not be calling this function outside `tune.run` or while using the class API. 


{'mse': 0.10689314042904505}


## HyperOpt

HyperOpt is a Bayesian optimization algorithm by [James Bergstra et al.](https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf)
  - [Home page](http://hyperopt.github.io/hyperopt/)
  - [GitHub](https://github.com/hyperopt/hyperopt)
  - [HyperOpt: Bayesian Hyperparameter Optimization](https://blog.dominodatalab.com/hyperopt-bayesian-hyperparameter-optimization/), Subir Mansukhani (2019)
    

In [16]:
NUM_SAMPLES=128

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# to limit number of cores, uncomment and set max_concurrent 
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,gamma,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_xgb_c3e257ac,TERMINATED,,0.7,0,0.0105391,0,4,435.406,0.000188888,9.90358,0.65,secrets/wandb.txt,True,iowa,1,9.36415,0.203631
my_xgb_c3e659ec,TERMINATED,,0.3,0,0.00673344,3,2,428.153,0.538855,2.32463,0.6,secrets/wandb.txt,True,iowa,1,6.59392,0.681993
my_xgb_c3eaaf4c,TERMINATED,,0.15,0,0.00133237,5,4,7988.18,0.00163546,19.9476,0.8,secrets/wandb.txt,True,iowa,1,577.132,0.114781
my_xgb_c3ef7d38,TERMINATED,,0.2,0,0.0187659,2,3,735.64,0.000262686,32.5338,0.6,secrets/wandb.txt,True,iowa,1,12.3839,0.11581
my_xgb_c3f3c654,TERMINATED,,0.6,0,0.00155207,0,3,4032.08,0.0246711,0.00107456,0.8,secrets/wandb.txt,True,iowa,1,291.457,0.124603
my_xgb_c3f805d4,TERMINATED,,0.15,0,0.00106609,1,5,2700.39,0.0127785,0.0424791,0.8,secrets/wandb.txt,True,iowa,1,37.4302,0.661069
my_xgb_c3fa751c,TERMINATED,,0.4,0,0.00556109,3,2,238.15,0.000451262,43.2731,0.75,secrets/wandb.txt,True,iowa,1,3.10039,3.25644
my_xgb_c3fe9fac,TERMINATED,,0.55,0,0.0154611,0,1,5605.97,0.00272955,4.48938,0.7,secrets/wandb.txt,True,iowa,1,411.645,0.109036
my_xgb_c4023978,TERMINATED,,0.5,0,0.00126552,5,3,4734.92,0.00967981,33.1173,0.65,secrets/wandb.txt,True,iowa,1,594.542,0.1519
my_xgb_c405e10e,TERMINATED,,0.75,0,0.00153686,2,2,678.644,0.306625,1.59856,0.9,secrets/wandb.txt,True,iowa,1,12.9699,4.08285


Start Time           2020-10-14 04:50:19.825404
End Time             2020-10-14 05:43:47.462255
0:53:27


In [17]:
analysis.results_df.columns

Index(['mse', 'time_this_iter_s', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore', 'experiment_tag',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight',
       'config.subsample', 'config.colsample_bytree', 'config.reg_alpha',
       'config.reg_lambda', 'config.gamma', 'config.learning_rate',
       'config.wandb.project', 'config.wandb.api_key_file',
       'config.wandb.log_config'],
      dtype='object')

In [18]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
c419415e,0.106830,2020-10-14_04-55-31,208.694009,4374,4,3,0.75,0.10,0.009577,1.219649,0,0.023078
c54737c0,0.106832,2020-10-14_05-20-22,385.872504,7221,3,3,0.65,0.15,0.013007,1.561307,0,0.009488
c54ceee0,0.106860,2020-10-14_05-16-19,119.204720,5468,3,4,0.80,0.05,0.072716,2.172529,0,0.007060
c5fe2124,0.107370,2020-10-14_05-32-01,164.177733,6130,3,4,0.80,0.05,0.024101,5.281423,0,0.012514
c6047ff6,0.107403,2020-10-14_05-33-57,216.857344,5835,3,4,0.85,0.05,0.035173,8.033476,0,0.007173
...,...,...,...,...,...,...,...,...,...,...,...,...
c5024138,4.676598,2020-10-14_05-12-07,83.853531,487,6,0,0.75,0.60,0.825219,0.002500,0,0.001854
c41d6cb6,4.762144,2020-10-14_04-55-22,1.424998,110,4,2,0.60,0.35,0.228840,17.810306,0,0.008214
c67b12c4,6.050190,2020-10-14_05-40-33,23.376860,639,6,5,0.80,0.70,0.274691,0.000181,0,0.001009


In [19]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    


In [20]:
best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.1, gamma=0, gpu_id=None,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.023077837289899438, max_delta_step=None,
             max_depth=4, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=4374, n_jobs=-1,
             num_parallel_tree=None, random_state=42,
             reg_alpha=0.009577332554566264, reg_lambda=1.2196486740226578,
             scale_pos_weight=None, subsample=0.75, tree_method=None,
             validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106830 (STD 0.0120)
Raw CV RMSE 18770.443039 (STD 2229.3542)


## Optuna

HyperOpt is a Bayesian optimization algorithm by [Takuya Akiba, et al.](https://arxiv.org/abs/1907.10902)

 - [Home](https://optuna.org/)
 - [Using Optuna to Optimize XGBoost Hyperparameters](https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407), Crissman Loomis, 2020



In [21]:
# optuna
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

# identical tune args
tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.quniform(0, 6, 1),
        'min_child_weight': tune.quniform(0, 6, 1),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True,
            "name": get_random_tag(6)
        }           
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_optuna",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),                    
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bytree,learning_rate,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,iter,total time (s),mse
my_xgb_61a8a6b0,TERMINATED,,0.8,0.0149737,0,4,120.227,0.0560079,0.000111923,0.5,1,8.96587,1.89341
my_xgb_61a9ddfa,TERMINATED,,0.45,0.0679413,2,1,3896.1,0.0648505,4.4491,0.75,1,431.654,0.113042
my_xgb_61ad00c0,TERMINATED,,0.15,0.0190601,5,4,4831.85,0.000791337,0.00010736,0.4,1,386.274,0.108627
my_xgb_61ae2fa4,TERMINATED,,0.45,0.0295256,3,6,3686.22,0.0154152,3.36107,0.85,1,553.914,0.110057
my_xgb_61b0bcba,TERMINATED,,0.8,0.00129091,0,4,1433.24,0.000258045,0.134315,0.55,1,91.1411,1.82097
my_xgb_61b3e69c,TERMINATED,,0.65,0.00150306,2,2,122.458,0.036849,0.553708,0.55,1,2.65759,9.59974
my_xgb_61b66200,TERMINATED,,0.7,0.0126552,4,0,6767.16,0.0193134,0.0436491,0.75,1,1719.88,0.110289
my_xgb_61b78ac2,TERMINATED,,0.55,0.0176795,2,5,3049.82,0.000134547,22.0891,0.7,1,347.081,0.110866
my_xgb_61baa252,TERMINATED,,0.3,0.00194973,2,0,2332.49,0.583871,0.00114846,0.75,1,167.371,0.17917
my_xgb_61bd2248,TERMINATED,,0.55,0.00754911,5,5,6447.08,0.000398651,6.13819,0.85,1,1506.99,0.110973


Start Time           2020-10-14 05:44:52.694050
End Time             2020-10-14 08:31:37.200526
2:46:44


In [22]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df

,mse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.min_child_weight,config.subsample,config.colsample_bytree,config.reg_alpha,config.reg_lambda,config.gamma,config.learning_rate
trial_id,,,,,,,,,,,,
62529814,0.106621,2020-10-14_06-29-19,94.348070,892,4,2.0,0.70,0.15,0.005971,0.000204,0,0.024089
6324f430,0.107068,2020-10-14_07-45-15,879.607330,9154,6,3.0,0.65,0.15,0.027771,0.050165,0,0.003316
62d8851e,0.107118,2020-10-14_07-16-32,138.973085,3964,4,6.0,0.80,0.10,0.001332,0.517925,0,0.016578
6265f760,0.107563,2020-10-14_06-47-27,831.376425,4328,4,0.0,0.60,0.70,0.022878,0.103568,0,0.025073
6209e5d8,0.107717,2020-10-14_06-10-35,188.853099,2438,5,0.0,0.70,0.15,0.060495,0.152984,0,0.017384
...,...,...,...,...,...,...,...,...,...,...,...,...
62fd8ab2,9.085646,2020-10-14_07-25-07,24.646753,110,5,1.0,0.55,0.60,0.150842,0.084836,0,0.002165
61b3e69c,9.599742,2020-10-14_05-45-11,2.657587,122,4,2.0,0.55,0.65,0.036849,0.553708,0,0.001503
62f53cb8,9.901193,2020-10-14_07-23-00,15.404225,146,6,2.0,0.40,0.30,0.007525,0.068154,0,0.001043


In [23]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.15000000000000002,
             gamma=0, gpu_id=None, importance_type='gain',
             interaction_constraints=None, learning_rate=0.024089088973297788,
             max_delta_step=None, max_depth=4, min_child_weight=2.0,
             missing=nan, monotone_constraints=None, n_estimators=892,
             n_jobs=-1, num_parallel_tree=None, random_state=42,
             reg_alpha=0.005970653012091714, reg_lambda=0.0002044422616092076,
             scale_pos_weight=None, subsample=0.7000000000000001,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.106621 (STD 0.0135)
Raw CV RMSE 18735.160425 (STD 2491.9345)


# LightGBM with HyperOpt

In [11]:
def my_lgbm(config):
    
    # fix these configs 
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    config['num_leaves'] = 2 + int(config['num_leaves'])
    
    lgbm = LGBMRegressor(objective='regression',
                         num_leaves=config['num_leaves'],
                         learning_rate=config['learning_rate'],
                         n_estimators=config['n_estimators'],
                         max_bin=200,
                         bagging_fraction=config['bagging_fraction'],
                         feature_fraction=config['feature_fraction'],
                         feature_fraction_seed=7,
                         min_data_in_leaf=2,
                         # these are to suppress warnings
                         colsample_bytree=None,
                         min_child_samples=None,
                         subsample=None,
                         verbose=-1,
                         # early stopping params, maybe in fit
                         #early_stopping_rounds=early_stopping_rounds,
                         #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                         #num_boost_round=num_boost_round,
                         )
    
    scores = np.sqrt(-cross_val_score(lgbm, df[predictors], df[response],
                                      scoring="neg_mean_squared_error",
                                      cv=kfolds))
    tune.report(mse=np.mean(scores))
    return {'mse': np.mean(scores)}

In [ ]:
# tune LightGBM
print("LightGBM")
#!conda install -y -c conda-forge lightgbm

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,n_estimators,num_leaves,wandb/api_key_file,wandb/log_config,wandb/project,iter,total time (s),mse
my_lgbm_8c56eafc,PENDING,,0.763567,0.724284,0.0350065,4493.91,4,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c5a459e,PENDING,,0.786611,0.33843,0.00614739,5322.25,6,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c5e0530,PENDING,,0.714983,0.475004,0.00110012,6200.59,6,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c61dec6,PENDING,,0.510418,0.325655,0.00454235,328.779,6,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c6604c4,PENDING,,0.588352,0.0195669,0.00295786,2368.28,5,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c69d96e,PENDING,,0.669557,0.210609,0.00596761,3580.3,2,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c6d8988,PENDING,,0.627906,0.241588,0.00156139,3275.96,2,secrets/wandb.txt,True,iowa,,,
my_lgbm_8be06dfa,RUNNING,,0.714805,0.46834,0.00716233,9727.47,9,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c241294,RUNNING,,0.65758,0.194515,0.00173311,3914.6,9,secrets/wandb.txt,True,iowa,,,
my_lgbm_8c2c2b46,RUNNING,,0.611485,0.562155,0.00232134,8650.59,2,secrets/wandb.txt,True,iowa,,,


# LightGBM with Optuna

In [23]:
# tune LightGBM
print("LightGBM")
#!conda install -y -c conda-forge lightgbm

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,n_estimators,num_leaves,iter,total time (s),mse
my_lgbm_7d3455d6,PENDING,,0.638715,0.0906994,0.00128929,552.143,7,,,
my_lgbm_7d3577f4,PENDING,,0.522545,0.717897,0.0325693,201.083,5,,,
my_lgbm_7d36882e,PENDING,,0.558216,0.0599268,0.0432801,3732.14,0,,,
my_lgbm_7d379be2,PENDING,,0.751411,0.712717,0.0777035,3482.93,8,,,
my_lgbm_7d38d430,PENDING,,0.796536,0.167995,0.00404344,2878.28,8,,,
my_lgbm_7d39fa22,PENDING,,0.527068,0.45742,0.0358343,738.443,1,,,
my_lgbm_7d3b0250,PENDING,,0.604765,0.336884,0.0239656,2140.19,7,,,
my_lgbm_7d20efc8,RUNNING,,0.702808,0.351058,0.0364129,9934.31,2,,,
my_lgbm_7d274b98,RUNNING,,0.634296,0.474335,0.00190683,5508.01,9,,,
my_lgbm_7d287856,RUNNING,,0.625376,0.57177,0.00500009,5734.38,7,,,


(pid=20035) [LightGBM] [Warning] feature_fraction is set=0.571770341840848, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.571770341840848
(pid=20035) [LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2
(pid=20035) [LightGBM] [Warning] bagging_fraction is set=0.6253764589739184, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6253764589739184
(pid=20230) [LightGBM] [Warning] feature_fraction is set=0.7170614234344085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7170614234344085
(pid=20230) [LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2
(pid=20230) [LightGBM] [Warning] bagging_fraction is set=0.6102957995924482, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6102957995924482
(pid=20253) [LightGBM] [Warning] feature_fraction is set=0.09069940369902647, colsample_

KeyboardInterrupt: 

(pid=20249) [LightGBM] [Warning] feature_fraction is set=0.1270795950493495, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.1270795950493495
(pid=20249) [LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2
(pid=20249) [LightGBM] [Warning] bagging_fraction is set=0.5658704190777574, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5658704190777574


In [25]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.num_leaves', 'config.bagging_fraction',
       'config.feature_fraction', 'config.learning_rate']].sort_values('mse')
analysis_results_df

,mse,date,time_this_iter_s,config.n_estimators,config.num_leaves,config.bagging_fraction,config.feature_fraction,config.learning_rate
trial_id,,,,,,,,
b801c570,0.105969,2020-10-14_08-40-33,6.645187,2758,5,0.754804,0.049949,0.027701
b81f315a,0.106189,2020-10-14_08-40-58,23.128222,1742,10,0.517380,0.055519,0.020289
b883d6be,0.106252,2020-10-14_08-45-27,6.423838,2715,5,0.700356,0.043747,0.026954
b926ea48,0.106657,2020-10-14_08-50-26,4.533699,1758,5,0.699876,0.039295,0.045055
ba643cb2,0.106706,2020-10-14_09-00-16,4.103416,1734,5,0.712548,0.047620,0.043218
...,...,...,...,...,...,...,...,...
b8afc8fa,0.323659,2020-10-14_08-45-33,1.587285,247,11,0.724945,0.102919,0.001350
b9724bf0,0.344946,2020-10-14_08-51-17,8.306199,153,11,0.725653,0.170354,0.001309
baf8e042,0.346311,2020-10-14_09-02-16,20.323498,155,11,0.681775,0.646190,0.001111


In [26]:
num_leaves = analysis_results_df.iloc[0]['config.num_leaves']
bagging_fraction = analysis_results_df.iloc[0]['config.bagging_fraction']
feature_fraction = analysis_results_df.iloc[0]['config.feature_fraction']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'num_leaves': num_leaves,
    'bagging_fraction': bagging_fraction,
    'feature_fraction': feature_fraction,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

lgbm = LGBMRegressor(objective='regression',
                     num_leaves=best_config['num_leaves'],
                     learning_rate=best_config['learning_rate'],
                     n_estimators=best_config['n_estimators'],
                     max_bin=200,
                     bagging_fraction=best_config['bagging_fraction'],
                     feature_fraction=best_config['feature_fraction'],
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

LGBMRegressor(bagging_fraction=0.7548044328777966,
              feature_fraction=0.04994854048380451, feature_fraction_seed=7,
              learning_rate=0.027701226718815856, max_bin=200,
              min_data_in_leaf=2, n_estimators=2758, num_leaves=5,
              objective='regression', verbose=-1)

Log1p CV RMSE 0.105969 (STD 0.0126)
Raw CV RMSE 18612.385681 (STD 2325.3205)


# Ray Cluster

- Clusters are defined in `ray1.1.yaml`
- boto3 and AWS CLI configured credentials are used, so [install and configure AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html)
- Edit `ray1.1.yaml` file with your region, availability zone, subnet, imageid information
    - to get those variables launch the latest Deep Learning AMI (Ubuntu 18.04) Version 35.0 into a small instance in your favorite region/zone
    - test that it works
    - note those 4 variables: region, availability zone, subnet, AMI imageid
    - terminate the instance and edit `ray1.1.yaml` accordingly
    - in future you can create your own image with everything pre-installed and specify its AMI imageid, instead of using the generic image and installing everything at launch.
- To run the cluster: 
`ray up ray1.1.yaml`
    - Creates head instance using image specified.
    - Installs ray and related requirements
    - Clones this Iowa repo
    - Launches worker nodes per auto-scaling parameters (currently we fix the number of nodes because we're not benching the time the cluster will take to auto-scale)
- After cluster starts you can check AWS console and note that several instances launched.
- Check `ray monitor ray1.1.yaml` for any error messages
- Run Jupyter on the cluster with port forwarding
 `ray exec ray1.1.yaml --port-forward=8899 'source ~/anaconda3/bin/activate tensorflow_p36 && jupyter notebook --port=8899'`
- Open the notebook on the generated URL e.g. http://localhost:8899/?token=5f46d4355ae7174524ba71f30ef3f0633a20b19a204b93b4
- Make sure to hoose the default kernel to make sure it runs in the conda environment with all installs
- Make sure to use the ray.init() command given in the startup messages.
- You can also run a terminal on the head node of the cluster with
 `ray attach /Users/drucev/projects/iowa/ray1.1.yaml`
- You can also ssh explicitly with the IP address and the generated private key
 `ssh -o IdentitiesOnly=yes -i ~/.ssh/ray-autoscaler_1_us-east-1.pem ubuntu@54.161.200.54`
- run port forwarding to the Ray dashboard with   
`ray dashboard ray1.1.yaml`
and then open
 http://localhost:8265/
- the cluster will incur AWS charges so `ray down ray1.1.yaml` when complete

see https://docs.ray.io/en/latest/cluster/launcher.html for additional info

In [ ]:
# make sure local ray is shutdown
ray.shutdown()


In [ ]:
# launch cluster in terminal
# initialize ray on cluster
ray.init(address='localhost:6379', _redis_password='5241590000000000')


In [ ]:
# run hyperopt/xgb on cluster
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        "max_depth": tune.randint(0, 6),
        'min_child_weight': tune.randint(0, 6),
        "subsample": tune.quniform(0.4, 0.9, 0.05),
        "colsample_bytree": tune.quniform(0.05, 0.8, 0.05),
        "reg_alpha": tune.loguniform(1e-04, 1),
        "reg_lambda": tune.loguniform(1e-04, 100),
        "gamma": 0,
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_xgb,
                    name="xgb_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1, 
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.max_depth', 'config.min_child_weight', 'config.subsample',
       'config.colsample_bytree', 'config.reg_alpha', 'config.reg_lambda', 'config.gamma',
       'config.learning_rate']].sort_values('mse')
analysis_results_df


In [ ]:
max_depth = analysis_results_df.iloc[0]['config.max_depth']
min_child_weight = analysis_results_df.iloc[0]['config.min_child_weight']
subsample = analysis_results_df.iloc[0]['config.subsample']
colsample_bytree = analysis_results_df.iloc[0]['config.colsample_bytree']
reg_alpha = analysis_results_df.iloc[0]['config.reg_alpha']
reg_lambda = analysis_results_df.iloc[0]['config.reg_lambda']
reg_gamma = analysis_results_df.iloc[0]['config.gamma']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'reg_alpha': reg_alpha,
    'reg_lambda': reg_lambda,
    'gamma': reg_gamma,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


In [ ]:
#lgbm on cluster
print("LightGBM")

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

tune_kwargs = {
    "num_samples": NUM_SAMPLES,
    "config": {
        "n_estimators": tune.loguniform(100, 10000),
        'num_leaves': tune.randint(0, 10),
        "bagging_fraction": tune.uniform(0.5, 0.8),
        "feature_fraction": tune.uniform(0.01, 0.8),
        "learning_rate": tune.loguniform(0.001, 0.1),
        "wandb": {
            "project": "iowa",
            "api_key_file": "secrets/wandb.txt",
            "log_config": True
        }    
    }
}

analysis = tune.run(my_lgbm,
                    name="lgbm_hyperopt",
                    metric="mse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
                    raise_on_failed_trial=False, # otherwise no reults df returned if any trial error                    
                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                    **tune_kwargs)

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


In [ ]:
analysis_results_df = analysis.results_df[['mse', 'date', 'time_this_iter_s',
       'config.n_estimators', 'config.num_leaves', 'config.bagging_fraction',
       'config.feature_fraction', 'config.learning_rate']].sort_values('mse')
analysis_results_df


In [ ]:
num_leaves = analysis_results_df.iloc[0]['config.num_leaves']
bagging_fraction = analysis_results_df.iloc[0]['config.bagging_fraction']
feature_fraction = analysis_results_df.iloc[0]['config.feature_fraction']
learning_rate = analysis_results_df.iloc[0]['config.learning_rate']
N_ESTIMATORS = analysis_results_df.iloc[0]['config.n_estimators']    

best_config = {
    'num_leaves': num_leaves,
    'bagging_fraction': bagging_fraction,
    'feature_fraction': feature_fraction,
    'learning_rate': learning_rate,
    'n_estimators':  N_ESTIMATORS
}

lgbm = LGBMRegressor(objective='regression',
                     num_leaves=best_config['num_leaves'],
                     learning_rate=best_config['learning_rate'],
                     n_estimators=best_config['n_estimators'],
                     max_bin=200,
                     bagging_fraction=best_config['bagging_fraction'],
                     feature_fraction=best_config['feature_fraction'],
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

## Concluding remarks

We observe a modest but non-negligeable improvement in the target metric with a less manual process vs. sequential tuning.
hyoperopt.optuna yes
ray maybe, depends on further work compare w/optuna and hyperopt online. depends on integration between ray, early stopping and search algo
cluster , in general no need and costs add up. MacBook Pro w/16 threads and desktop with GPU are plenty.

I intend to use HyperOpt and Optuna for xgboost going forward, no more grid search for me! In every case I've applied them, I've gotten at least a small improvement in the best metrics I found using grid search methods. Additionally, it's fire and forget (although with a little elbow grease the 4-pass sequential grid search could be made fire and forget.)

These two algorithms seem to be the most popular but I may try the other algos systematically. 

I am surprised that Elasticnet, i.e. regularized linear regression outperforms boosting. 
This dataset has been heavily engineered so that linear methods work well. Predictors were chosen using lasso/elasticnet and we used log and Box-Cox transforms to force predictors to follow assumptions of least-squares.  

This tends to validate one of the [critiques of machine learning](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3624052), that the most powerful ML methods don't necessarily converge all the way to the best solution. If you have a ground truth that is linear plus noise, a complex XGBoost or neural network algorithm should get arbitrarily close to the closed-form optimal solution but will never match it exactly. XGBoost is piecewise constant and the complex neural network is subject to the vagaries of stochastic gradient descent. 

But Elasticnet with L1 + L2 regularization plus gradient descent and hyperparameter optimization is still machine learning. It's simply the form best matched to the problem. In the real world where things don't match assumptions of least-squares, boosting performs extremely well. And even on this dataset, engineered for the linear models, SVR and KernelRidge performed better than Elasticnet (not shown) and ensembling Elasticnet with XGBoost, LightGBM, SVR, neural networks worked best of all. 

To paraphrase Casey Stengel, clever feature engineering will always outperform clever model algorithms and vice-versa<sup>*</sup>. But improving your hyperparameters with these best practices will always improve your results.

<sup>*</sup>This is not intended to make sense.
